In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import re
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.metrics.pairwise import linear_kernel

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
netflix_data = pd.read_csv('/kaggle/input/netflix-shows/netflix_titles.csv')
netflix_data.head()

In [ ]:
netflix_data = netflix_data[['type', 'title', 'listed_in', 'description']]

netflix_data['listed_in'] = [re.sub(r'[^\w\s]', ' ', t) for t in netflix_data['listed_in']]
netflix_data['description'] = [re.sub(r'[^\w\s]', ' ', t) for t in netflix_data['description']]

netflix_data['listed_in'] = [t.lower() for t in netflix_data['listed_in']]
netflix_data['description'] = [t.lower() for t in netflix_data['description']]

In [ ]:
netflix_data['combined'] = netflix_data['listed_in'] + ' ' + netflix_data['title'] + ' ' + netflix_data['description']
netflix_data.drop(["description","listed_in","type"],axis=1,inplace=True)
netflix_data.head()

In [ ]:
vectorizer = TfidfVectorizer()
matrix = vectorizer.fit_transform(netflix_data['combined'])
cosine_similarities = linear_kernel(matrix,matrix)
movie_title = netflix_data['title']
indices = pd.Series(netflix_data.index, index=netflix_data['title'])

In [ ]:
def content_recommender(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_similarities[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31]
    movie_indices = [i[0] for i in sim_scores]
    
    return movie_title.iloc[movie_indices]

In [ ]:
content_recommender('3%').to_csv('content_based.csv',index = False)